In [ ]:
import pandas as pd
import numpy as np

import time
import spacy
import re

import sys

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import itertools

from random import getrandbits
from random import seed

In [ ]:
#import et merge des datasets
data_lyrics= pd.DataFrame()
for i in range(6):
    try: #si mon fichier existe
        print(i)
        data_lyrics= pd.concat([data_lyrics, 
                                pd.read_csv("data/0{}_dataset_lyrics.csv".format(i+1))])
    except:
        #au cas ou , si mon fichier n'existe pas alors j'ajoute un df vide
        data_lyrics= pd.concat([data_lyrics, pd.DataFrame()])
        
data_lyrics= data_lyrics.rename({"test":"Lyrics"}, axis=1)

In [ ]:
print(data_lyrics.shape)
# print(data_lyrics.head(1))
print(data_lyrics["Artiste"].unique())
print(data_lyrics["Style"].unique()) #erreur dnas l'écriture de la modalité
#Rock Alternatif
dico_map= {"Rock\\Alternatif":"Rock/Alternatif",
           "Rock/Alternatif":"Rock/Alternatif",
          "metal":"Metal",
          "pop":"Pop",
           "rap":"Rap",
          }
data_lyrics["Style"]= data_lyrics["Style"].map(dico_map)
data_lyrics["Style"].unique()

In [ ]:
data_lyrics= data_lyrics.drop("Url", axis= 1)
data_lyrics.columns

In [ ]:
data_lyrics.head()
#Necessité de traiter les lyrics ==> Quand on a un featuring, le chanteur qui chante 
#est reperé grace au i`AUTOR NAME`i

In [ ]:
#on va creer un dico pour associer à chaque artiste son style
#le style reste constant dans le temps
dico_artiste_style= {artiste: data_lyrics[data_lyrics["Artiste"] == artiste]\
                     .iloc[0]["Style"]
                     for artiste in data_lyrics["Artiste"].unique()}
print(dico_artiste_style)

#on peut drop la colonne
data_lyrics= data_lyrics.drop("Style", axis= 1)
data_lyrics.reset_index(inplace= True, drop= True)
data_lyrics.columns

In [ ]:
test_lyrics= data_lyrics["Lyrics"][9]
pattern_i= re.compile('i[\S]*i\s')
pattern_i.sub("",test_lyrics)
test_lyrics

In [ ]:
#traitement des None
def verification(df):
    """
    Me permet de vérifier les quantités de valeurs nulles dans mes colonnes
    """
    print("Vérifier si des valeurs sont None")
    for col in df.columns:
        nul= (df[col].isnull()).sum()
        sh= df.shape[0]
        print("\n")
        print(col)
        print(nul)
        print("Soit: {}% de valeurs None".format((nul/sh)*100))
    pass

verification(data_lyrics)
print(data_lyrics.shape)

#drop les lignes ou les lyrics manquent
data_lyrics= data_lyrics[data_lyrics["Lyrics"].notnull()]
verification(data_lyrics)
data_lyrics.shape

## Evolution du nombre de mot unique par auteur

In [ ]:
#nombre de musique pour chaque auteur et par année
data_lyrics[["Artiste", "Annee", "Lyrics"]].groupby(["Artiste", "Annee"]).agg(["count"])

In [ ]:
#multi index dataframe
data_multi= data_lyrics.set_index(["Artiste"])\
.sort_values(["Artiste"])

In [ ]:
data_multi.head(10)

In [ ]:
test= data_multi.groupby(["Artiste", "Annee"])["Lyrics"].agg(" ".join).to_frame()
test["compteur"]= test["Lyrics"].apply(lambda x: len(set(x.split(" "))))
test.reset_index(level=1, inplace=True)
test.head(10)

In [ ]:
#evolution du nombres de mots uniques par artiste par an
def plot_per_artist_per_year(df, col, titre="Evolution du nombre de mots uniques par an"):
    """
    Permet de plot l'évolution par artiste par an d'une métrique souhaité, comme le nombre de mots
    uniques ou bien le nombre de mot unique par musique en moyenne
    -df: le df doit être de type ==> index Artiste, 1 colonne Annee sort, et la colonne 
        de notre métrique
    -col: c'est le nom de la métrique que l'on retrouve dans df
    """
    seed(10)
    artiste_name= list(test.index.get_level_values('Artiste').unique())

    fig= plt.figure(figsize=(8,5))
    sns.set(style="darkgrid")
    cycled= itertools.cycle('bgrcmykwy')

    for art in artiste_name:
        year= list(df.query("Artiste == '{}'".format(art))["Annee"])
        data= list(df.query("Artiste == '{}'".format(art))[col])
    #     print(data)
    #     print(year)
        line=""
        if bool(getrandbits(1)):
            line="--"
        else:
            line="-"
        plt.plot(year, data, label= art, color= next(cycled), linestyle= line)


    #pour faire sortir la legende du graphique
    plt.title(titre, fontsize= 15)
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.show()
    pass

plot_per_artist_per_year(test, "compteur")

### Remarque

Les rappeurs sont largement au dessus des autres artistes en terme de nombre de mots uniques par année

Peut-être existe t-il un lien entre nombre de musique et le nombre de mots uniques.

In [ ]:
nb_song_per_year= data_lyrics[["Artiste", "Annee", "Lyrics"]]\
                    .groupby(["Artiste", "Annee"])\
                    .count()

nb_song_per_year= nb_song_per_year.reset_index(level=1).sort_index().drop("Annee", axis= 1)
nb_song_per_year

per_song= pd.concat([test, nb_song_per_year], axis= 1)
per_song["mot_unique_per_song"]= per_song.apply(lambda row: row[2]/row[3], axis= 1)

plot_per_artist_per_year(per_song, "mot_unique_per_song")
per_song.head(10)

## Remarque


## Evolution du nombre de mot unique par style

In [ ]:
#on va faire la même mais par style de musique
#on va laisser Artiste à la place de style pour faciliter le copier coller
data_lyrics_style= data_lyrics.copy()
data_lyrics_style["Artiste"]= data_lyrics["Artiste"].map(dico_artiste_style)
data_lyrics_style[["Artiste", "Annee", "Lyrics"]].groupby(["Artiste", "Annee"]).agg(["count"])

#multi index dataframe
data_multi_style= data_lyrics_style.set_index(["Artiste"])\
.sort_values(["Artiste"])

test= data_multi_style.groupby(["Artiste", "Annee"])["Lyrics"].agg(" ".join).to_frame()
test["compteur"]= test["Lyrics"].apply(lambda x: len(set(x.split(" "))))
test.reset_index(level=1, inplace=True)
test.head(10)

In [ ]:
plot_per_artist_per_year(test, "compteur")

## Remarque

In [ ]:
nb_song_per_year_style= data_lyrics_style[["Artiste", "Annee", "Lyrics"]]\
                    .groupby(["Artiste", "Annee"])\
                    .count()

nb_song_per_year_style= nb_song_per_year_style.reset_index(level=1)\
                    .sort_index().drop("Annee", axis= 1)
nb_song_per_year_style
per_song_style= pd.concat([test, nb_song_per_year_style], axis= 1)
per_song_style["mot_unique_per_song"]= per_song_style.apply(lambda row: row[2]/row[3], axis= 1)

plot_per_artist_per_year(per_song_style, "mot_unique_per_song")

## Remarque

## Cherchons à analyser quel style de musique est le plus vulgaire

On va donc utiliser [une liste de ban-word](https://www.freewebheaders.com/bad-words-list-and-page-moderation-words-list-for-facebook/) qui est utilisé pour la modération sur Facebook pour définir quel style est le plus vulgaire

In [ ]:
nono_word= pd.read_csv("data/ban_word.csv", header= None, names= ["nono"])
nono_word= nono_word.drop(index= 0).reset_index(drop= True)
nono_word.head()

In [ ]:
ban_word_list= nono_word["nono"].tolist()

In [ ]:
data_lyrics_nono= data_lyrics.copy()
data_lyrics_nono["Artiste"]= data_lyrics["Artiste"].map(dico_artiste_style)
data_lyrics_nono[["Artiste", "Annee", "Lyrics"]].groupby(["Artiste", "Annee"]).agg(["count"])

#multi index dataframe
data_multi_nono= data_lyrics_nono.set_index(["Artiste"])\
.sort_values(["Artiste"])

test= data_multi_nono.groupby(["Artiste", "Annee"])["Lyrics"].agg(" ".join).to_frame()
test["compteur"]= test["Lyrics"].apply(lambda x: len(set(x.split(" "))))
test.reset_index(level=1, inplace=True)
test.head(10)

In [ ]:
#à peine long
test["num_nono_word"]= test.apply(lambda x: len([word for word in x[1].split(" ")\
                                         if word in ban_word_list]), axis= 1)

In [ ]:
test.head()

In [ ]:
test["proportion_nono"]= test.apply(lambda x: x[3]/(x[3]+x[2])
                                    , axis= 1)
test.head()

In [ ]:
#on commence par un plot simple de l'évolution du nombre de nono word par style de musique
plot_per_artist_per_year(test, "num_nono_word", titre= "Evolution de la vulgarité en nombre de mot")

#puis on va évaluer le nombre de nono moyen par style par musique
nb_song_per_year_style= data_lyrics_style[["Artiste", "Annee", "Lyrics"]]\
                    .groupby(["Artiste", "Annee"])\
                    .count()

nb_song_per_year_style= nb_song_per_year_style.reset_index(level=1)\
                    .sort_index().drop("Annee", axis= 1)
nb_song_per_year_style
per_song_style= pd.concat([test, nb_song_per_year_style], axis= 1)
per_song_style["nono_per_song"]= per_song_style.apply(lambda row: row[3]/row[5], axis= 1)

plot_per_artist_per_year(per_song_style, "nono_per_song", titre= "Evolution de la vulgarité par nombre de musique publié par année")

# enfin on observe l'évolution de la proportion de gros mot dans notre lyrics (par mot unique)
plot_per_artist_per_year(test, "proportion_nono", titre= "Evolution de la vulgarité en % de mot unique")